In [ ]:
import pandas as pd
import numpy as np
import os
import re


import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy

from imblearn.over_sampling import SMOTE



from datetime import datetime
from pandas_profiling import ProfileReport


In [ ]:
# definitions 
datain_path = 'data/'

explorations_path = 'explorations/'
if not os.path.exists(explorations_path): 
    os.makedirs(explorations_path)

In [ ]:
datasets = {
    'train': 'Train.xlsx',
    'test':'Test.xlsx', 
    'both': {
        'train': 'Train.xlsx',   
        'test':'Test.xlsx',
    } 
}

#datasets = pd.DataFrame(datasets, columns=['name', 'path']).set_index('name')

dataset_name = 'train'


In [ ]:
if dataset_name == 'both': 
    data = pd.DataFrame()
    for dataset_path in datasets[dataset_name].values(): 
        tmp = pd.read_excel(os.path.join(datain_path, dataset_path))
        data = pd.concat([data, tmp])

else:    
    dataset_path = datasets[dataset_name]
    data = pd.read_excel(os.path.join(datain_path, dataset_path))
data.head()

In [ ]:
data.isna().sum()

# Explorations

## profile report

profile = ProfileReport(
    data,
    title='Raw data',
    minimal=False, 
    correlations={
    "pearson": {"calculate": True},
    "spearman": {"calculate": False},
    "kendall": {"calculate": False},
    "phi_k": {"calculate": False},
    "cramers": {"calculate": False},
    }
)
profile.to_file(os.path.join(explorations_path, 'profile_data_raw.html'))


In [ ]:
n_feature=10

def get_feature_imp_by_expl(data, base_col, n_feature=n_feature): 
    
    # make sure every combination of levels exist, fill with 0 if no obs
    base = data[base_col].unique()
    Income = [0, 1]
    idx = pd.MultiIndex.from_product(
        [base, Income],
        names=[base_col, 'Income']
    )

    pd1 = pd.DataFrame(index=idx)
    

    a = data.groupby([base_col, 'Income']).size().to_frame().rename(columns={0:'nobs'})
    
    a = pd.concat([pd1, a], axis=1)
    a.loc[a.nobs.isna(), 'nobs'] = 0
    
    a['nobs_rel'] = a.groupby(level=base_col).transform(lambda x: x / (x[0] + x[1]))
    value_cols = a.columns.to_list()
    a.reset_index(inplace=True)
    
    # top Features by nobs: 
    topFeat = data.groupby(base_col).size().to_frame().rename(columns={0:'nobs'})\
        .sort_values('nobs', ascending=False).iloc[0:n_feature,:]\
        .index.to_list()
    
    a.sort_values(['nobs', base_col], ascending=False, inplace=True)
    
    #print('len(topFeat)', len(topFeat))
    #print('len(a)', len(a))
    #print('n_feature', n_feature)
    #print('a[base_col].nunique()', a[base_col].nunique())

    if len(topFeat) < a[base_col].nunique(): 
        print(f'***Features Filtered to top_{n_feature} by nobs!***')
    #print(a[base_col].nunique())

    a = a.loc[a[base_col].isin(topFeat),:]
    return a, value_cols

def plot_feature_imp_by_expl(data, base_col): 
    
    a, value_cols = get_feature_imp_by_expl(data, base_col)
    n_plots = len(value_cols) + 1

    fig, ax = plt.subplots(ncols = n_plots  , figsize=(20,7), gridspec_kw={'width_ratios': [3,3,1]})
    for i, col in enumerate(value_cols): 
        sns.barplot(data=a, x=base_col, y=col, hue='Income', ax=ax[i])#.set_title(col) # [0:10]
        ax[i].tick_params(labelrotation=45)

    sns.countplot(data=data, x='Income', ax=ax[n_plots-1])
    plt.show()
    
def get_target_ratio(data):  
    a = data.groupby(['Income']).size().to_frame().rename(columns={0:'nobs'})
    a['nobs_rel'] = a.transform(lambda x: x / (x[0] + x[1]))
    value_cols = a.columns.to_list()
    a.reset_index(inplace=True)
    return a.loc[a.Income == 1, 'nobs_rel'].to_list()[0]


def get_feature_imp_by_target_ratio(data, base_col, weighted=False): 

    target_ratio = get_target_ratio(data)
    target_ratio
                                            
    a, _ = get_feature_imp_by_expl(data, base_col, n_feature=100)
    
    #########
    nObsPerFeatClass =  data.groupby([base_col]).size().to_frame().rename(columns={0:'nobs'})
                           

    ratio_per_level = a.loc[a.Income == 1, [base_col,'nobs_rel']]\
        .set_index(base_col)\
        .rename(columns={'nobs_rel':'class1_ratio'})

    ratio_per_level = pd.concat([ratio_per_level, nObsPerFeatClass], axis=1)
    #min_max_scaler_obs = MinMaxScaler()
    ratio_per_level['nobs_rel'] = ratio_per_level.nobs / sum(ratio_per_level.nobs)


    
    ratio_per_level['diff_to_target'] = ratio_per_level['class1_ratio'] - target_ratio
    ratio_per_level['diff_to_target_dir'] = ['neg' if obs < 0 else 'pos' for obs in ratio_per_level['diff_to_target']]
    
    if weighted: 
        weights = np.power(ratio_per_level['nobs_rel'], 1./3)
    else: 
        weights = 1
        
    ratio_per_level['diff_to_target_abs'] = abs(ratio_per_level['diff_to_target']) * weights
    
    #print(ratio_per_level)

    ratio_per_level.sort_values('diff_to_target_abs', ascending=False, inplace=True)
    ratio_per_level['diff_to_target_abs_cumsum'] = ratio_per_level.diff_to_target_abs.cumsum()
    ratio_per_level


    x = ratio_per_level['diff_to_target_abs_cumsum'].values.reshape(-1, 1) #df.values #returns a numpy array
    min_max_scaler = MinMaxScaler()
    ratio_per_level['diff_to_target_abs_cumsum_scaled'] = min_max_scaler.fit_transform(x)

    print('TargetClass1_ratio', target_ratio)
    print(ratio_per_level[['diff_to_target_dir', 'diff_to_target_abs']])

    return ratio_per_level


def plot_feature_imp_by_target_ratio(data, base_col, weighted=False): 

    r = get_feature_imp_by_target_ratio(data, base_col, weighted)

    sns.lineplot(data=r, y=r.index, x='diff_to_target_abs_cumsum_scaled')
    plt.show()
    
    
def plot_feature_imp_by_tree(data, base_col, n_feature=n_feature): 
    # prepare
    onehot = OneHotEncoder()
    X_train_cat = data.loc[:,[base_col]]
    #X_train_cat = data[base_col]

    X_train_onehot = onehot.fit_transform(X_train_cat)
    X_train_onehot_df = pd.DataFrame(X_train_onehot.toarray(), columns=onehot.get_feature_names())
    X_train_onehot_df

    X_train_onehot_df = pd.get_dummies(data[base_col], prefix=base_col)

    # train
    dt_gini = DecisionTreeClassifier(random_state = 1)
    X_train = X_train_onehot_df#.drop(columns=['x0_Africa','x0_Europe', 'x0_Oceania'])
    y_train = data.Income


    dt_gini.fit(X_train, y_train) # data[base_col]
    print('Score:', dt_gini.score(X_train, y_train))

    #dt_gini.feature_importances_
    #tree.plot_tree(dt_gini)

    #plt.barh(onehot.get_feature_names(), dt_gini.feature_importances_)

    #print(dt_gini.feature_importances_)
    sorted_idx = dt_gini.feature_importances_.argsort()#[0:10]
    plotdata = pd.DataFrame({
        'Feature': X_train.columns[sorted_idx], 
        'Importance': dt_gini.feature_importances_[sorted_idx]}).sort_values('Importance', ascending=False)
    #plt.barh()
    #print(plotdata)
    sns.barplot(data=plotdata.iloc[0:n_feature,:], x='Importance', y='Feature')
    plt.xlabel("Feature Importance")
    
    plt.show()

    
def plot_feature_imp(data, base_col, force_barplot=True, weighted=False): 
    print('Class distributions')
    if (data[base_col].nunique() < 6) | force_barplot:
        n_plots = 2
        plot_feature_imp_by_expl(data, base_col)
    else: 
        n_plots = 1
        
    print('\nElbow')        
    plot_feature_imp_by_target_ratio(data, base_col, weighted)
    
    print('\nDecision Tree')
    plot_feature_imp_by_tree(data, base_col)

In [ ]:
# 
data.info()

In [ ]:
# init 
cols_to_drop = []
cols_to_onehot = []

# prep
pred_config = {
    'cardinality': 'original' # low, medium, high, original
} 

cardinality = pred_config['cardinality']
print('cardinality:', cardinality)

error_log = {'cleaning': []}


In [ ]:
# extract gender from name?!
salutation = data.Name.str.split(' ', n=1, expand=True)[0]
if salutation.nunique() != 3: 
    raise ValueError('Unexpected levels of salutation')
    
print(salutation.value_counts())

#gender = ['male' if s == 'Mr.' else 'female' for s in salutation]
#data['gender'] = gender

male = [1 if s == 'Mr.' else 0 if s in ['Mrs.', 'Miss'] else np.nan for s in salutation]
data['male'] = male

if data.male.isna().sum() > 0: 
    raise Warning('NAs instroduced')


sns.countplot(data=data, hue=data.Income, x='male')#.set_title(col)
plt.show()
    
cols_to_drop.append('Name')


In [ ]:
# Compute age from Birthday

# clean whitespaces
data.Birthday = data.Birthday.str.replace(' ', '')
# define date format
dob_format = '%B%d,%Y'

# transform Birthday to datetime, catching the leap year error 

## helper fct to subtract one day from datetime if error occurs
def subone(obj):
    val = int(obj.group(0))
    return str(val-1)

## init and loop over dates
dob = []
warn_log = []
for i, d in enumerate(data.Birthday): 
    try: 
        dob.append(datetime.strptime(d, dob_format).date())

    except ValueError as e: 
        if str(e) == 'day is out of range for month': 
            dt = datetime.strptime(re.sub('\d{1,2}', subone, d, count=1), dob_format).date()
            warn_log.append((d, dt))
            dob.append(dt)
        else: 
            raise NotImplementedError('Do not know how to deal with that error!')
            dt = np.nan
            warn_log.append((d, dt))
            dob.append(dt)
        
# add age column 
data['age'] = [np.floor((datetime.strptime('2048-12-31', '%Y-%m-%d').date() - d).days / 365.2425) for d in dob]

# inspect
sns.histplot(data, x='age')
plt.show()
print('Min age:' , min(data.age))

# drop date col 
cols_to_drop.append('Birthday')


In [ ]:
data.info()

In [ ]:

# 'Native Continent' to bin 
base_col = 'Native Continent'
#sns.countplot(data=data, hue=data.Income, x=base_col)#.set_title(col)
#plt.show()

plot_feature_imp(data, base_col, weighted=False)

#low, medium, high, original
try: 
    if cardinality in ['low', 'medium']:
        target_col = 'from_europe_or_asia'
        #data['from_europe'] = [1 if a == 'Europe' else 0 for a in data[base_col]]
        data[target_col] = [1 if a in ['Europe', 'Asia'] else 0 for a in data[base_col]]
    elif cardinality == 'original':
        target_col = 'native_continent'
        data[target_col] = data[base_col]
        cols_to_onehot.append(target_col)
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)
    

sns.countplot(data=data, x=target_col, hue='Income')
plt.show()

cols_to_drop.append(base_col)



In [ ]:
# Marital Status
base_col = 'Marital Status'
#target_col = 'marital_status'

data[base_col].value_counts()

plot_feature_imp(data, base_col, weighted=False)

try: 
    if cardinality == 'low': 
        target_col = 'maritalStatus_married'
        data[target_col] = [1 if a in ['Married', 'Married - Spouse in the Army'] else 0 for a in data[base_col]]
        
    elif cardinality == 'medium': 
        target_col = 'maritalStatus'
        mapping = {
            'Married':'Married',
            'Single':'Single',
            'Divorced':'Divorced',
            'Separated':'Separated',
            'Widow':'Widow',
            'Married - Spouse Missing':'SpouseMissing',
            'Married - Spouse in the Army':'Married'
        }

        data[target_col] = data[base_col].map(mapping)
        cols_to_onehot.append(target_col)
        
    elif cardinality == 'original': 
        target_col = 'maritalStatus'
        mapping = {
            'Married':'Married',
            'Single':'Single',
            'Divorced':'Divorced',
            'Separated':'Separated',
            'Widow':'Widow',
            'Married - Spouse Missing':'SpouseMissing',
            'Married - Spouse in the Army':'MarriedArmy'
        }

        data[target_col] = data[base_col].map(mapping)
        cols_to_onehot.append(target_col)

    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)
    
    
#sns.countplot(data=data, x=target_col)
sns.countplot(data=data, x=target_col, hue='Income')
plt.show()

cols_to_drop.append(base_col)


In [ ]:
# Lives with
base_col = 'Lives with'
print(data[base_col].value_counts())
plot_feature_imp(data, base_col, weighted=False)

try: 
    if(cardinality == 'low'): 
        target_col = 'household_livesWithPartner'
        data[target_col] = [1 if a in ['Wife', 'Husband'] else 0 for a in data[base_col]]
    elif(cardinality == 'medium'): 
        target_col = 'household'
        mapping = {
            'Wife': 'Partner',
            'Other Family': 'Family',
            'Children': 'Children',
            'Alone': 'Alone',
            'Husband': 'Partner',
            'Other relatives': 'Family'
        }

        print(mapping)

        data[target_col] = data[base_col].map(mapping)
        cols_to_onehot.append(target_col)
    elif(cardinality == 'original'): 
        target_col = 'household'
        mapping = {
            'Wife': 'Wife',
            'Other Family': 'Family',
            'Children': 'Children',
            'Alone': 'Alone',
            'Husband': 'Husband',
            'Other relatives': 'Other'
        }

        print(mapping)

        data[target_col] = data[base_col].map(mapping)
        cols_to_onehot.append(target_col)
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)

sns.countplot(data=data, x=target_col, hue='Income')
plt.show()

cols_to_drop.append(base_col)


In [ ]:

# 'Base Area' to bin 
base_col = 'Base Area'
plot_feature_imp(data, base_col, weighted=False)

try: 
    if cardinality == 'low': 
        target_col = 'basearea_fanfoss' # basearea_northbury
        target_val = 'Fanfoss'
        target_col_alt = 'basearea_northbury'
        target_val_alt = 'Northbury'

        print('\nResult:')
        data[target_col] = [1 if a == target_val else 0 for a in data[base_col]]

        print('\nAlternative result:')
        test = data[['Income', base_col]].copy()
        test[target_col_alt] = [1 if a == target_val_alt else 0 for a in test[base_col]]
        sns.countplot(data=test, x=target_col_alt, hue='Income')
        plt.show()
    elif cardinality == 'medium':
        data[target_col] = [
            target_val if a == target_val 
            else target_val_alt if a == target_val_alt 
            else 'Rest' for a in data[base_col]]
        cols_to_onehot.append(target_col)

    elif cardinality == 'original':
        data[target_col] = data[base_col]
        cols_to_onehot.append(target_col)
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)

    
sns.countplot(data=data, x=target_col, hue='Income')
plt.show()

cols_to_drop.append(base_col)

In [ ]:
# Education Level 
base_col = 'Education Level'
target_col = 'education'
print(data.columns)
plot_feature_imp(data, base_col, weighted=False)


edu_mapping = pd.read_excel(os.path.join(datain_path, 'edu_mapping_2.xlsx'), 'Tabelle2')
mapping_options = ['level_0', 'level_1', 'numeric', 'original', 'low']


#low, medium, high, original
try: 
    if cardinality == 'low':
        m_option = mapping_options[4]
        plot_fct = sns.countplot
    elif cardinality == 'medium': 
        m_option = mapping_options[4]
        plot_fct = sns.countplot
    elif cardinality == 'high': 
        m_option = mapping_options[2]
        plot_fct = sns.histplot
    elif cardinality == 'original':
        m_option = mapping_options[3]
        plot_fct = sns.countplot       
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)
    


#print(data[base_col].value_counts())

#mapping = dict(edu_mapping[['name', mapping_options[2]]].set_index('name'))
#mapping = {k:v for k,v in edu_mapping[['name', mapping_options[2]]].set_index('name').items()}
#mapping = edu_mapping[['name', mapping_options[2]]].set_index('name')
mapping = edu_mapping[['name', m_option]].rename(columns={m_option: target_col})
print(mapping)

# drop if reruning the cell 
if target_col in data.columns: 
    data.drop(columns=[target_col], inplace=True)

data = data.merge(mapping, left_on=base_col, right_on='name', how='left')
data.drop(columns=['name'], inplace=True)  

# plot target col against prediction classes
fig, ax = plt.subplots(figsize=(15,7))
plot_fct(data=data, x=target_col, hue='Income', ax=ax)
#plt.xticks(rotation=45)
plt.show()

cols_to_drop.append(base_col)
cols_to_onehot.append(target_col)


data[[base_col, target_col]]

In [ ]:
# years of education 
base_col = 'Years of Education'
target_col = 'education_years'
data.rename(columns={base_col: target_col}, inplace=True)

data.head()
#sns.histplot(data=data, y=target_col)

In [ ]:
# Employment Sector
base_col = 'Employment Sector'
target_col = 'empl_sector'
plot_feature_imp(data, base_col, weighted=False)


print(data[base_col].value_counts())

#low, medium, high, original
try: 
    if cardinality == 'deprecated':
        mapping = {
            'Private Sector - Services ': 'private',
            'Self-Employed (Individual)': 'self',
            'Public Sector - Others': 'public',
            '?': 'unknown',
            'Private Sector - Others': 'private',
            'Self-Employed (Company)': 'self',
            'Public Sector - Government': 'public',
            'Unemployed': 'delete',
            'Never Worked': 'delete'
            }

    elif cardinality in ['low', 'medium', 'original']: 
        mapping = {
            'Private Sector - Services ': 'private_services',
            'Self-Employed (Individual)': 'self_individual',
            'Public Sector - Others': 'public_others',
            '?': 'unknown',
            'Private Sector - Others': 'private_others',
            'Self-Employed (Company)': 'self_company',
            'Public Sector - Government': 'public_gov',
            'Unemployed': 'unemployed',
            'Never Worked': 'unemployed'
            }
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)

print(mapping)
    
data[target_col] = data[base_col].map(mapping)


fig, ax = plt.subplots(figsize=(15,7))
sns.countplot(data=data, x=target_col, hue='Income', ax=ax)
plt.show()

cols_to_drop.append(base_col)
cols_to_onehot.append(target_col)

In [ ]:
# role
base_col = 'Role'
target_col = 'empl_role'

plot_feature_imp(data, base_col, weighted=False)

#low, medium, high, original
try: 
    if cardinality in ['low', 'medium']:
        mapping = {
            'Professor': 'Professor',
            'Management': 'Management',
            'Repair & constructions': 'Operational_low',
            'Administratives': 'Operational',
            'Sales': 'Sales',
            'Other services': 'Services',
            'Machine Operators & Inspectors': 'Operational',
            '?': 'unknown',
            'Transports': 'Operational_low',
            'Cleaners & Handlers': 'Cleaners',
            'Agriculture and Fishing': 'Operational',
            'IT': 'IT_Security',
            'Security': 'IT_Security',
            'Household Services': 'Household',
            'Army': 'Operational_low'
        }
    elif cardinality == 'original':       
        mapping = {
            'Professor': 'Professor',
            'Management': 'Management',
            'Repair & constructions': 'Constructions',
            'Administratives': 'Administratives',
            'Sales': 'Sales',
            'Other services': 'Services',
            'Machine Operators & Inspectors': 'Operator',
            '?': 'unknown',
            'Transports': 'Transports',
            'Cleaners & Handlers': 'Cleaners',
            'Agriculture and Fishing': 'Agriculture',
            'IT': 'IT', 
            'Security': 'Security',
            'Household Services': 'Household',
            'Army': 'Army'
        }
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)

print(data[base_col].value_counts())

print(mapping)
    
data[target_col] = data[base_col].map(mapping)


fig, ax = plt.subplots(figsize=(15,7))
sns.countplot(data=data, x=target_col, hue='Income', ax=ax)
plt.show()

cols_to_drop.append(base_col)
cols_to_onehot.append(target_col)

In [ ]:
 

# Working Hours per week
base_col = 'Working Hours per week'
target_col = 'working_hrs_week'

data.rename(columns={base_col: target_col}, inplace=True)

sns.histplot(data=data, x=target_col, hue='Income', bins=30)
plt.show()

data.head()



In [ ]:
# Money Received
base_col = 'Money Received'
target_col = 'group_b_received_money'


#low, medium, high, original
try: 
    if cardinality == 'low':
        data[target_col] = [1 if v != 0 else 0 for v in data[base_col]]
    elif cardinality == 'original':
        data[target_col] = data[base_col]
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)

cols_to_drop.append(base_col)


sns.countplot(data=data, x=target_col, hue='Income')
plt.show()

#data[[base_col, target_col]]



In [ ]:
   

# Ticket Price
base_col = 'Ticket Price'
target_col = 'group_c_payed'

#low, medium, high, original
try: 
    if cardinality == 'low':
        data[target_col] = [1 if v != 0 else 0 for v in data[base_col]]
    elif cardinality == 'original':
        data[target_col] = data[base_col]
    else: 
        raise NotImplementedError(f'Can not interpret cardinality "{cardinality}" for base feature "{base_col}"!')
except Exception as e:
    error_log['cleaning'].append(e)
    raise Warning(e)
    

cols_to_drop.append(base_col)


sns.countplot(data=data, x=target_col, hue='Income')
plt.show()

#data[[base_col, target_col]]

In [ ]:
# Check for errors
for name, log in error_log.items():
    if len(log) > 0: 
        print(f'{name}:\n {log}')
        raise Warning('Errors occured! See above.')

In [ ]:
error_log

In [ ]:
# drop cols
cols_to_drop.append('CITIZEN_ID')
data.drop(columns=cols_to_drop, inplace=True)

In [ ]:
## profile report

create_cleaning_report = False
if create_cleaning_report: 
    profile = ProfileReport(
        data,
        title=f'Cleaned data {dataset_name}' ,
        minimal=False, 
        correlations={
        "pearson": {"calculate": True},
        "spearman": {"calculate": False},
        "kendall": {"calculate": False},
        "phi_k": {"calculate": False},
        "cramers": {"calculate": False},
        }
    )
    profile.to_file(os.path.join(explorations_path, f'profile_data_cleaned_{dataset_name}.html'))


# Explorations

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
# target distribution

sns.countplot(data=data, x='Income')
plt.show()

In [ ]:
# Prepare figure
fig = plt.figure(figsize=(10, 8))

# Obtain correlation matrix. Round the values to 2 decimal cases. Use the DataFrame corr() and round() method.
corr = np.round(data.corr(method="pearson"), decimals=2)

# Build annotation matrix (values above |0.5| will appear annotated in the plot)
mask_annot = np.absolute(corr.values) >= 0.5
annot = np.where(mask_annot, corr.values, np.full(corr.shape,"")) # Try to understand what this np.where() does

# Plot heatmap of the correlation matrix
sns.heatmap(data=corr, annot=annot, cmap=sns.diverging_palette(220, 10, as_cmap=True), 
            fmt='s', vmin=-1, vmax=1, center=0, square=True, linewidths=.5)

# Layout
fig.subplots_adjust(top=0.95)
fig.suptitle("Correlation Matrix", fontsize=20)

plt.savefig(os.path.join(explorations_path, 'correlation_matrix.png'), dpi=200)
plt.show()

In [ ]:
# distributions 

ncols = 4
n_plots = data.shape[1]
nrows = int(np.ceil(n_plots/ncols))



fig, ax = plt.subplots(ncols=ncols, nrows=nrows, figsize=(15,13))
col_no = 0
for i in range(nrows):
    for j in range(ncols): 
        if col_no < n_plots:
            col = data.columns[col_no]
            print(col)
            if data[col].dtype in [np.float, np.int]: 
                sns.histplot(data=data, hue=data.Income, x=col, ax=ax[i,j], bins=30).set_title(col)
            else : 
                sns.countplot(data=data, hue=data.Income, x=col, ax=ax[i,j], dodge=True).set_title(col)
            ax[i,j].tick_params(labelrotation=45)
            col_no +=1

fig.tight_layout()

plt.savefig(os.path.join(explorations_path, 'distributions.png'), dpi=200)
plt.show()


# Feature Engineering ideas
- age + household: age diff to mean of hh group
- 

### Imputations: 
- empl_sector == unkown

## onehot

In [ ]:
cols_to_onehot

In [ ]:


#one hot encode 
features = pd.get_dummies(data=data, columns=cols_to_onehot, drop_first=False)
features.info()

# Data Preparation

In [ ]:
# prep config

prep_config = {
    'overSampling': True, 
    'normalize': False,
    'outlier': False, # uni- / multivariate, working hours per week
    'feature_selection':'boruta'
}

## feature_selection

In [ ]:
X = features.copy().drop('Income', axis=1).values
y = features.copy().loc[:,'Income'].values

In [ ]:
# https://github.com/scikit-learn-contrib/boruta_py
# https://towardsdatascience.com/feature-selection-with-borutapy-f0ea84c9366


# load X and y
# NOTE BorutaPy accepts numpy arrays only, hence the .values attribute
# X = pd.read_csv('examples/test_X.csv', index_col=0).values
# y = pd.read_csv('examples/test_y.csv', header=None, index_col=0).values
# y = y.ravel()

if prep_config['feature_selection'] == 'boruta': 


    # define random forest classifier, with utilising all cores and
    # sampling in proportion to y labels
    rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

    # define Boruta feature selection method
    feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=1)

    # find all relevant features - 5 features should be selected
    feat_selector.fit(X, y)

    # check selected features - first 5 features are selected
    print(feat_selector.support_)
    # check ranking of features
    print(feat_selector.ranking_)

    # call transform() on X to filter it down to selected features
    X_filtered = feat_selector.transform(X)
    
    X_metadata = pd.DataFrame({
    'Features': (features.drop('Income', axis=1).columns.to_list()),
    'support': (feat_selector.support_), 
    'ranking': (feat_selector.ranking_)
    })

    X_metadata

In [ ]:
X_metadata.support.sum()

## upcaling to cope with class imbalance 

## Normalizing data 

# Modelling

In [ ]:
###Train test split on updated X
X_t, X_val, y_t, y_val = train_test_split(X, y, random_state=42, stratify=None)
sns.countplot(y_t)
plt.show()

if prep_config['overSampling']: 
    sm = SMOTE(random_state=2, n_jobs=-1, k_neighbors=5, sampling_strategy='auto')
    X_t, y_t = sm.fit_sample(X_t, y_t)
    sns.countplot(y_t)
    plt.show()

In [ ]:

###Instantiating Random Forest Classifier
rf2 = RandomForestClassifier(n_jobs=-1, n_estimators=500, oob_score=True, max_depth=6, random_state=42)
###Fitting Random Forest Classifier to train and test
rf2.fit(X_t, y_t)
###Predicting on test data
y_pred = rf2.predict(X_val)
###Test Score
rf2.score(X_val, y_val)
###Training Score
rf2.score(X_t, y_t)

In [ ]:
###Instantiating Random Forest Classifier
rf2 = RandomForestClassifier(n_jobs=-1, n_estimators=500, oob_score=True, max_depth=6, random_state=42)
###Fitting Random Forest Classifier to train and test
rf2.fit(X_t, y_t)
###Predicting on test data
predicted = rf2.predict(X_val)
###Test Score
rf2.score(X_val, y_val)
###Training Score
rf2.score(X_t, y_t)

print(classification_report(y_val, predicted))



In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(
    learning_rate=.1,
    random_state=0, verbose=False,
    subsample=1,
    n_estimators=1000, max_depth=3, min_impurity_decrease = 0.1,
    max_features='auto',
    n_iter_no_change=20)

clf.fit(X_t, y_t)
clf.predict(X_t[:2])
predicted = clf.predict(X_val)

print(classification_report(y_val, predicted))

clf.score(X_val, y_val)


In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html#sphx-glr-auto-examples-model-selection-plot-learning-curve-py
import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, axes=None, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate 3 plots: the test and training learning curve, the training
    samples vs fit times curve, the fit times vs score curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    axes : array of 3 axes, optional (default=None)
        Axes to use for plotting the curves.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:

          - None, to use the default 5-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    if axes is None:
        _, axes = plt.subplots(1, 3, figsize=(20, 5))

    axes[0].set_title(title)
    if ylim is not None:
        axes[0].set_ylim(*ylim)
    axes[0].set_xlabel("Training examples")
    axes[0].set_ylabel("Score")

    train_sizes, train_scores, test_scores, fit_times, _ = \
        learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs,
                       train_sizes=train_sizes,
                       return_times=True)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    fit_times_mean = np.mean(fit_times, axis=1)
    fit_times_std = np.std(fit_times, axis=1)

    # Plot learning curve
    axes[0].grid()
    axes[0].fill_between(train_sizes, train_scores_mean - train_scores_std,
                         train_scores_mean + train_scores_std, alpha=0.1,
                         color="r")
    axes[0].fill_between(train_sizes, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1,
                         color="g")
    axes[0].plot(train_sizes, train_scores_mean, 'o-', color="r",
                 label="Training score")
    axes[0].plot(train_sizes, test_scores_mean, 'o-', color="g",
                 label="Cross-validation score")
    axes[0].legend(loc="best")

    # Plot n_samples vs fit_times
    axes[1].grid()
    axes[1].plot(train_sizes, fit_times_mean, 'o-')
    axes[1].fill_between(train_sizes, fit_times_mean - fit_times_std,
                         fit_times_mean + fit_times_std, alpha=0.1)
    axes[1].set_xlabel("Training examples")
    axes[1].set_ylabel("fit_times")
    axes[1].set_title("Scalability of the model")

    # Plot fit_time vs score
    axes[2].grid()
    axes[2].plot(fit_times_mean, test_scores_mean, 'o-')
    axes[2].fill_between(fit_times_mean, test_scores_mean - test_scores_std,
                         test_scores_mean + test_scores_std, alpha=0.1)
    axes[2].set_xlabel("fit_times")
    axes[2].set_ylabel("Score")
    axes[2].set_title("Performance of the model")

    return plt


In [ ]:

title = r"Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"
# SVC is more expensive so we do a lower number of CV iterations:
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
estimator = clf
plot_learning_curve(estimator, title, X_t, y_t, ylim=(0.7, 1.01),
                    cv=cv, n_jobs=-1)

plt.show()

In [ ]:
import xgboost as xgb

clf = xgb.XGBClassifier(
    use_label_encoder=False, 
    eval_metric='logloss'
)
clf.fit(X_t, y_t)
predicted = clf.predict(X_val)

print(classification_report(y_val, predicted))

clf.score(X_val, y_val)
